<a href="https://colab.research.google.com/github/NikolasGialitsis/GrowthCurveCovid19/blob/master/Gialitsis_GrowthCurveCovid19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

========================================================
========================================================
## Machine Learning Methods in Computational Biology  22/3/2020
### Assignment 1 
### @Author Gialitsis Nikos DS2.0005
========================================================
========================================================
----------------------------------------------------------------

Task (100  points): identify  the optimal model  from  a given simple parametric  family (polynomials  of  degree  up  to  M=9 as in Bishop’s book)  that  best  characterizes  the growth curve of COVID-19 cases without overfitting the data(using Occam’s razor principle). You should use time series data for different countries,including (but not limited to) 
China, SouthKorea, Japan,Italy, Spain,  France, UK, Germany, Netherlands,USA,Greece. 

To train candidate models for a country, you can use a proportion(say 2/3) of randomly selected data points for days up to date X= March 20, 2020. Then you can test the models’ performanceusing the rest of the data points not used for the training. The model that minimizes the test set  error is considered optimal. You may use the BIC criterion for model selection.  After the optimal model has been determined, you should also test its future predictions for the cases beyond date X. Date X should be a parameter,so that you can reuse your code to produce results considering any date beyond March 20, 2020 as date X. Of course, growth curves of the cases are not enough to show how the pandemic is spreading since there is evidence that a lot  of infected people remain asymptomatic, spreading the virus without  knowing it, and  this  is  why strict  social  distancing  is  a survival  attitude.  How  could  someone  estimate  the actual number  of all cases  from  what  we  currently  measure (using  PCR-based tests)? Review  the  related literature and present your opinion (in no more than half page with a clear justification).